# [실습 1] Built-in & Third-party Tool 사용하기

학습 내용:
1. 환경 설정: 필요한 라이브러리 설치 및 API 키 설정
2. Third-party Tool: Tavily 웹 검색 도구 사용법
3. Built-in Tool: LangChain에서 기본 제공하는 DuckDuckGo 검색 도구 사용법
4. Toolkit: 여러 도구를 한번에 가져오는 Toolkit 활용법 (Wikipedia)
5. Agent 생성: 정의된 도구들을 활용하여 스스로 추론하고 답을 찾는 Agent 만들기

## 1. 환경 설정

In [1]:
# 먼저 필요한 라이브러리들을 설치합니다.
# !pip install -qU langchain langchain_openai langchain-community langchain-tavily tavily-python duckduckgo-search wikipedia

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
# LangChain 관련 기본 import
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain import hub
from langchain.agents import create_openai_functions_agent, AgentExecutor

# LLM 모델을 초기화합니다. 이번 실습에서는 'gpt-4o-mini'를 사용합니다.
llm = ChatOpenAI(
    model = "gpt-4o-mini", # e.g. "openai/gpt-4o-mini", "openai/gpt-5"
    temperature=0
)

# 2. Third-party Tool: Tavily 웹 검색
- Tavily는 AI Agent를 위해 특별히 설계된 검색 엔진입니다.  
- 일반 검색 엔진보다 더 정확하고 간결한 정보를 제공하여 Agent의 성능을 높여줍니다.

**Tavily API Key 발급 방법**
1. Tavily 공식 홈페이지 접속: 우측 상단의 "Sign in" 또는 "Sign up"을 클릭하여 회원가입을 진행합니다. Google 계정으로 쉽게 가입할 수 있습니다.
공식 사이트 : https://auth.tavily.com/u/login/identifier?state=hKFo2SBETnBjX3pFNy1ubEpzQ0Jtemo5dnhLSm5DTjVsd2pGSKFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIGh5ZWVYbm5TLW5DS2pwbEF4aDJITGhvSElFMTRmQTg3o2NpZNkgUlJJQXZ2WE5GeHBmVFdJb3pYMW1YcUxueVVtWVNUclE

2. API Key 확인: 회원가입 후 대시보드(Dashboard)로 이동하면 "API Key" 섹션이 있습니다. 여기에 표시된 API 키를 복사합니다.

3. 무료 사용: Tavily는 "Researcher" 플랜으로 월 1,000건의 무료 API 호출을 제공하므로, 실습용으로는 충분합니다.

In [1]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [3]:
from langchain_tavily import TavilySearch

# Tavily 검색 도구를 생성합니다.
# max_results는 검색 결과의 최대 개수를 지정합니다.
tavily_tool = TavilySearch(
    max_results=5
)

In [6]:
# Tavily 도구를 직접 호출하여 테스트해볼 수 있습니다.
print("--- Tavily 검색 테스트 ---")
search_result = tavily_tool.invoke({"query": "LangChain이란 무엇인가요?"})
print(search_result)
print("-" * 20)

--- Tavily 검색 테스트 ---
{'query': 'LangChain이란 무엇인가요?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://cloud.google.com/use-cases/langchain?hl=ko', 'title': 'LangChain이란 무엇인가요? 예시 및 정의', 'content': 'LangChain은 대규모 언어 모델(LLM)로 애플리케이션을 더 쉽게 빌드할 수 있도록 지원하는 오픈소스 조정 프레임워크입니다. LLM을 다양한 데이터 소스에 연결하는 도구와', 'score': 0.9255605, 'raw_content': None}, {'url': 'https://www.ibm.com/kr-ko/think/topics/langchain', 'title': 'LangChain이란 무엇인가요?', 'content': 'LangChain은 거의 모든 LLM을 위한 일반 인터페이스 역할을 하며 LLM 애플리케이션을 구축하고 이를 외부 데이터 소스 및 소프트웨어 워크플로와 통합할 수 있는 중앙 집중식 개발 환경을 제공합니다. LangChain은 챗봇, 지능형 검색, 질문 답변, 요약 서비스뿐만 아니라 로봇 프로세스 자동화가 가능한 가상 에이전트와 같은 LLM 및 자연어 처리(NLP)에 이르기까지 대부분의 사용 사례를 지원할 수 있습니다. BigScience의 BLOOM, Meta AI의 LLaMa, Google의 Flan-T5와 같은 많은 오픈 소스 모델은 Hugging Face(ibm.com 외부 링크)를 통해 액세스할 수 있습니다. LangChain은 기본 제공 파운데이션 모델에만 국한되지 않으며, *CustomLLM* 클래스(ibm.com 외부 링크)를 통해 사용자 지정 LLM 래퍼를 사용할 수 있습니다. 마찬가지로 LangChain 통합 기능이 포함된 IBM watsonx API 및 Python SDK를 통해 *Watson

# 3. Built-in Tool: DuckDuckGo 웹 검색
- LangChain은 다양한 내장 도구를 제공합니다. DuckDuckGo는 그 중 하나로,
- 별도의 API 키 없이 사용할 수 있는 검색 도구입니다.

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun

# DuckDuckGo 검색 도구를 생성합니다.
duckduckgo_tool = DuckDuckGoSearchRun()

In [8]:
# DuckDuckGo 도구를 직접 호출하여 테스트해봅니다.
print("--- DuckDuckGo 검색 테스트 ---")
search_result_ddg = duckduckgo_tool.run("LangChain이란 무엇인가요?")
print(search_result_ddg)
print("-" * 20)

--- DuckDuckGo 검색 테스트 ---
Jul 20, 2025 · LangChain 主体分为 6 个模块，分别是对（大语言）模型输入输出的管理、外部数据接入、链的概念、（上下文记忆）存储管理、智能代理以及回调系统，通过文档的组织结构，你可以清晰 … 这算得上是一个非常惊艳的功能。 主要优点是，你用自然语言提出问题，LangChain把这个问题丢给LLMs，LLMs返回SQL脚本，然后LangChain会自动执行查询语句得到结果后，再丢给LLMs生成一 … 目前我把应用开发框架分成三类，一类是类似 langchain、llamaindex、semantic-kernel 这种偏整体流程设计、搭建，需要开发大量代码的。 一类是类似dify、fastgpt 这种轻代码，依靠 UI 界面上的编排 … LangChain的技术架构思路 不过，由于LangChain诞生时间较早，在2022年末，GPT-3.5刚诞生之时，开发者对大模型的想象主要是希望用其搭建一个又一个工作流，而这也成为LangChain的核心技术 … Langchain的学习路径 1、从官方文档入手，掌握LangChain的基本概念和使用方法。 2、了解LangChain的6大主要模块。 3、通读官方给出的示例，对于各个模块有更加直观的认识。 4、找一 …
--------------------


# 4. Toolkit: File Management Toolkit
- Toolkit은 특정 작업과 관련된 여러 도구의 묶음입니다.
https://python.langchain.com/v0.1/docs/integrations/toolkits/

In [9]:
from langchain_community.agent_toolkits import FileManagementToolkit

# 파일 관리 툴킷 초기화
fm_toolkit = FileManagementToolkit(root_dir="./my_agent_data")

In [10]:
# Toolkit에서 제공하는 도구들을 가져옵니다.
# get_tools() 메서드는 Toolkit에 포함된 모든 도구의 리스트를 반환합니다.
fm_tools = fm_toolkit.get_tools()
print(f"--- File Management Toolkit 도구 목록 ({len(fm_tools)}개) ---")
for tool in fm_tools:
    print(f"도구 이름: {tool.name}, 설명: {tool.description}")
print("-" * 20)

--- File Management Toolkit 도구 목록 (7개) ---
도구 이름: copy_file, 설명: Create a copy of a file in a specified location
도구 이름: file_delete, 설명: Delete a file
도구 이름: file_search, 설명: Recursively search for files in a subdirectory that match the regex pattern
도구 이름: move_file, 설명: Move or rename a file from one location to another
도구 이름: read_file, 설명: Read file from disk
도구 이름: write_file, 설명: Write file to disk
도구 이름: list_directory, 설명: List files and directories in a specified folder
--------------------


# 5. Agent 생성 및 실행

이제 위에서 정의한 도구들을 Agent에게 부여하여 실제 문제를 해결하게 해보겠습니다.

In [11]:
# Agent가 사용할 도구 목록을 정의합니다.
# 각 Tool 객체는 이름(name), 실행할 함수(func), 그리고 설명(description)으로 구성됩니다.
# 이 'description'은 Agent가 어떤 상황에 이 도구를 사용해야 할지 판단하는 매우 중요한 기준이 됩니다.

tools = [
    Tool(
        name="Tavily-Search",
        func=tavily_tool.invoke,
        description="최신 정보나 시사, 인물, 사실 관계 확인 등 웹 검색이 필요할 때 사용합니다. 다른 도구로 해결할 수 없는 질문에 대한 최종적인 답변 수단으로 유용합니다.",
    ),
    Tool(
        name="DuckDuckGo-Search",
        func=duckduckgo_tool.run,
        description="Tavily 검색 결과가 만족스럽지 않거나, 다른 관점의 웹 검색이 필요할 때 사용합니다.",
    ),
    # Wikipedia Toolkit의 도구들을 리스트에 추가합니다.
    *fm_tools
]

In [12]:
# Agent를 만들기 위한 프롬프트를 가져옵니다.
# hub.pull은 LangChain Hub에서 미리 만들어진 프롬프트 객체를 다운로드하는 기능입니다.
# "hwchase17/openai-functions-agent"는 OpenAI의 함수 호출 기능에 최적화된 표준 프롬프트입니다.
prompt = hub.pull("hwchase17/openai-functions-agent")

In [13]:
# 프롬프트 내용을 확인해볼 수 있습니다.
print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [14]:
# OpenAI Functions Agent를 생성합니다.
# 이 Agent는 LLM의 함수 호출(Function Calling) 기능을 사용하여 어떤 도구를 언제 호출할지 결정합니다.
agent = create_openai_functions_agent(llm, tools, prompt)

In [15]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.mess

In [16]:
dir(agent)

['InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__

In [17]:
# AgentExecutor는 Agent의 실행을 담당합니다.
# Agent가 도구를 호출하고, 그 결과를 다시 Agent에게 전달하는 등의 순환적인 과정을 관리합니다.
# verbose=True로 설정하면 Agent의 생각의 흐름(Chain of Thought)을 자세히 볼 수 있습니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
# 이제 Agent에게 질문을 던져보겠습니다.
# Agent는 질문을 이해하고, 가장 적절한 도구를 선택하여 실행하고, 최종 답변을 생성합니다.
print("\n--- Agent 실행 예시 1 ---")
question1 = "Attention Is All You Need 논문의 저자는 누구인가요? 그들의 소속도 알려주세요."
response1 = agent_executor.invoke({"input": question1})
print("\n[최종 답변]:", response1["output"])


--- Agent 실행 예시 1 ---


> Entering new AgentExecutor chain...
"Attention Is All You Need" 논문의 저자는 다음과 같습니다:

1. Ashish Vaswani
2. Noam Shazeer
3. Niki Parmar
4. Jakob Uszkoreit
5. Llion Jones
6. Aidan N. Gomez
7. Lukasz Kaiser
8. Stephan Gouws
9. Yoshua Bengio
10. Geoffrey Hinton

이 논문은 Google Brain 팀의 연구원들이 주로 작성하였으며, 저자들은 Google의 다양한 부서에 소속되어 있습니다.

> Finished chain.

[최종 답변]: "Attention Is All You Need" 논문의 저자는 다음과 같습니다:

1. Ashish Vaswani
2. Noam Shazeer
3. Niki Parmar
4. Jakob Uszkoreit
5. Llion Jones
6. Aidan N. Gomez
7. Lukasz Kaiser
8. Stephan Gouws
9. Yoshua Bengio
10. Geoffrey Hinton

이 논문은 Google Brain 팀의 연구원들이 주로 작성하였으며, 저자들은 Google의 다양한 부서에 소속되어 있습니다.


In [19]:
print("\n--- Agent 실행 예시 2 ---")
question2 = "OpenAI의 gpt-4o-mini 모델과 구글의 Gemini 1.5 Flash 모델을 비교 설명해줘."
response2 = agent_executor.invoke({"input": question2})
print("\n[최종 답변]:", response2["output"])


--- Agent 실행 예시 2 ---


> Entering new AgentExecutor chain...

Invoking: `Tavily-Search` with `OpenAI GPT-4o-mini vs Google Gemini 1.5 Flash 모델 비교`


{'query': 'OpenAI GPT-4o-mini vs Google Gemini 1.5 Flash 모델 비교', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://docsbot.ai/models/compare/gpt-4o-mini/gemini-1-5-flash-8b', 'title': 'GPT-4o Mini vs Gemini 1.5 Flash-8B', 'content': "GPT-4o Mini vs Gemini 1.5 Flash-8B - Detailed Performance & Feature Comparison GPT-4o Mini vs Gemini 1.5 Flash-8B Get a detailed comparison of AI language models OpenAI's GPT-4o Mini and Google's Gemini 1.5 Flash-8B, including model features, token pricing, API costs, performance benchmarks, and real-world capabilities to help you choose the right LLM for your needs. 4.   GPT-4o Mini vs Gemini 1.5 Flash-8B  GPT-4o Mini, developed by OpenAI, features a context window of 128K tokens and can generate up to 16.4K tokens in a single request. Compare costs for input and output

In [20]:
print("\n--- Agent 실행 예시 3 ---")
# Wikipedia 도구와 웹 검색 도구를 함께 사용해야 하는 복합적인 질문
question3 = "파일 'answer.txt'를 만들고, 그 안에 대한민국의 수도가 어디인지 검색해서 답을 써줘."
response3 = agent_executor.invoke({"input": question3})
print("\n[최종 답변]:", response3["output"])


--- Agent 실행 예시 3 ---


> Entering new AgentExecutor chain...

Invoking: `Tavily-Search` with `대한민국의 수도`


{'query': '대한민국의 수도', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://ko.wikipedia.org/wiki/%ED%95%9C%EA%B5%AD%EC%9D%98_%EC%88%98%EB%8F%84', 'title': '한국의 수도', 'content': '| 고구려 기원전 37년~668년 | 수도명 | 현재 위치 | | 수도 기간 |  | | 후백제 892년~936년 | 수도 | 현재 위치 | | 수도 기간 | | 일제강점기 1910년 8월 29일 ~1945년 8월 15일 | 수도 | 현재 위치 | 수도 기간 |  | | 경성부 京城府 | 대한민국 서울특별시 | 1910년 9월 30일~1945년 8월 15일 | | 대한민국 임시정부 1919년~1945년 | 임시 수도 | 현재 위치 | | 임시 수도 기간 |  | | 대한민국 1948년 8월 15일~ | 수도 | 수도 기간 |  | | 서울특별시 | 1953년 7월 27일~현재 | | 조선민주주의인민공화국 1948년 9월 9일~ | 수도명 | 수도 기간 |  | | 평양직할시 平壤直轄市 | 1950년 12월 5일~현재 | ↑ 조선총독부령 제7호 (메이지 43년 10월 1일 공포)', 'score': 0.8684817, 'raw_content': None}, {'url': 'https://m.kin.naver.com/qna/dirs/111001/docs/478501757?d1id=11', 'title': '대한민국 수도 서울 - 네이버 지식iN', 'content': '대한민국 수도 서울 대한민국의 수도인 서울이 어떻게 수도가 됐는지가 궁금합니다. 서울이 조선시대 때 한양이라 불리우며 조선의 수도였던 